# Prepare imports and Fetch data

In [49]:
from openai import OpenAI
import json
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
import requests
import tiktoken
from dotenv import load_dotenv

load_dotenv()

True

# Getting the Data

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Q1. Running Elasticsearch

In [5]:
!curl localhost:9200

{
  "name" : "7b372f5ee8f8",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "XcVGs4M1TmKcMbA4xowRDA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


The `version.build_hash` is `42f05b9372a9a4a470db3b52817899b99a76ee73`

# Q2. Indexing the data

In [8]:
es_client = Elasticsearch('http://localhost:9200') 
index_name = "course-questions"


In [ ]:

def index_docs(docs):
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }


    es_client.indices.create(index=index_name, body=index_settings)
    for doc in tqdm(docs):
        es_client.index(index=index_name, document=doc)

The function used to add data in elasticsearch is `index`

In [7]:
index_docs(docs=documents)

100%|██████████| 948/948 [00:02<00:00, 355.99it/s]


# Q3. Searching

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

In [9]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [15]:
response['hits']['hits'][0]['_score']

84.050095

# Q4. Filtering




Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

How do I debug a docker container?
How do I copy files from a different folder into docker container’s working directory?
How do Lambda container images work?
How can I annotate a graph?


In [19]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [24]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
search_results = [res['_source'] for res in response['hits']['hits']]
prompt = build_prompt(query, search_results)

In [26]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [27]:
len(prompt)

1462

# Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

In [39]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [44]:
len(encoding.encode(prompt))

322

In [47]:
encoding.encode(prompt)[0]

63842

In [45]:
encoding.decode_single_token_bytes(63842)

b"You're"

# Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?

In [50]:
client = OpenAI()
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [51]:
answer = llm(prompt)
print(answer)

To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container by using the `docker ps` command. Then, use the `docker exec` command with the `-it` option followed by the container ID and the command you wish to execute. Here's a step-by-step guide:

1. Identify the running container's ID using:
   ```
   docker ps
   ```

2. Execute the desired command in the running container. For example, to start a bash session:
   ```
   docker exec -it <container-id> bash
   ```
Replace `<container-id>` with the actual ID of your running container.

This will start an interactive bash session within the specified Docker container.


# Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens
You can redo the calculations with the values you got in Q6 and Q7.

In [56]:
print(f'{150_000 * 0.005/1000}$')

0.75$


In [55]:
print(f'{250_000 * 0.015/1000}$')

3.75$


## Based on my prompt length and answer length the numbers are as follows

In [57]:
prompt_len = len(encoding.encode(prompt))
answer_len = len(encoding.encode(answer))

In [58]:
print(f'input cost: {prompt_len*1000 * 0.005/1000}$\toutput cost: {answer_len * 1000 * 0.015/1000}$')

input cost: 1.61$	output cost: 2.34$
